# Apresentação

- PSR maintenance model
    - Formulação
    - Implementação
    - Comparação
    - Caso com dados reais
- Convolução
    - Soma de variáveis aleatórias
    - Implementação para 2 usinas
    - Implementação para n usinas
    - Método de Monte Carlo
    - Restrição de manutenção e vários estágios